In [ ]:
UDF’s are the most expensive operations hence use them only you have no choice and when essential.
UDF’s are used to extend the functions of the framework and re-use these functions on multiple DataFrame’s. For example, 
you wanted to convert every first letter of a word in a name string to a capital case; PySpark build-in features don’t 
have this function hence you can create it a UDF and reuse this as needed on many Data Frames. UDF’s are once created 
they can be re-used on several DataFrame’s and SQL expressions.

UDFs are a black box to PySpark hence it can’t apply optimization and you will lose all the optimization PySpark does on 
Dataframe/Dataset. When possible you should use Spark SQL built-in functions as these functions provide optimization. 
Consider creating UDF only when the existing built-in SQL function doesn’t have it.

In [ ]:
def udf_name(Df):
    code to perform task
    return output

 UDF is expensive


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

#Converting function to pyspark UDF 
convertUDF = udf(lambda z: convertCase(z))
# The default type of the udf() is StringType

df.select(col("Seqno"),convertUDF(col("Name")).alias("Name") ).show(truncate=False)

#Using UDF with PySpark DataFrame withColumn()
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z),StringType())    

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))).show(truncate=False)

#Creating UDF using annotation
@udf(returnType=StringType()) 
def upperCase(str):
    return str.upper()

df.withColumn("Cureated Name", upperCase(col("Name"))).show(truncate=False) 

#Using UDF on SQL - In order to use convertCase() function on PySpark SQL, you need to register the function with PySpark by using spark.udf.register().
spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE").show(truncate=False)
     
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " +
          "where Name is not null and convertUDF(Name) like '%John%'").show(truncate=False)     

#Its always best practice to check for null inside a UDF function rather than checking for null outside.
#In any case, if you can’t do a null check in UDF at lease use IF or CASE WHEN to check for null and call UDF conditionally.
#null check

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2").show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " +
          " where Name is not null and _nullsafeUDF(Name) like '%John%'").show(truncate=False)  

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+

+-----+-----------+
|Seqno|Name       |
+-----+---